## Extract the text

In [3]:
import whisper
# from tool_language_video_translate.utils import extract_audio_from_video
from tool_language_video_translate import utils


In [4]:
import os
video_name = "Pep_Guardiola_explained_why_Chelsea_is_so_good"
video_path = f"./video_data/{video_name}.mp4"
os.path.exists(video_path)
#/home/davidtito/Documents/Projects/tool_language_video_translate/video_data/Pep_Guardiola_explained_why_Chelsea_is_so_good.mp4

True

In [5]:
video_name = "Pep_Guardiola_explained_why_Chelsea_is_so_good"
video_path = f"./video_data/{video_name}.mp4"
audio_path = f"./video_data/origin_audio_{video_name}.wav"
utils.extract_audio(video_path=video_path, audio_path=audio_path)

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14 (GCC)
  configuration: --prefix=/usr --bindir=/usr/bin --datadir=/usr/share/ffmpeg --docdir=/usr/share/doc/ffmpeg --incdir=/usr/include/ffmpeg --libdir=/usr/lib64 --mandir=/usr/share/man --arch=x86_64 --optflags='-O2 -flto=auto -ffat-lto-objects -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wno-error=incompatible-pointer-types -Wp,-U_FORTIFY_SOURCE,-D_FORTIFY_SOURCE=3 -Wp,-D_GLIBCXX_ASSERTIONS -specs=/usr/lib/rpm/redhat/redhat-hardened-cc1 -fstack-protector-strong -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -m64 -march=x86-64 -mtune=generic -fasynchronous-unwind-tables -fstack-clash-protection -fcf-protection -mtls-dialect=gnu2 -fno-omit-frame-pointer -mno-omit-leaf-frame-pointer ' --extra-ldflags='-Wl,-z,relro -Wl,--as-needed -Wl,-z,pack-relative-relocs -Wl,-z,now -specs=/usr/lib/rpm/redhat/redhat-hardened-ld -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -Wl,--build-id

'./video_data/origin_audio_Pep_Guardiola_explained_why_Chelsea_is_so_good.wav'

In [6]:
def transcribe_audio(audio_path, model_size="small"):
    """
    Transcribe audio using whisper.
    """
    model = whisper.load_model(model_size)
    result = model.transcribe(audio_path)
    return result["text"]

In [7]:
text = transcribe_audio(audio_path)

In [8]:
text

" Why Chelsea plays so good? Because they have three centre defender closers, the two-holding midfielders, the relating move closers, the pocket closers, they have structured the five and two players so, so close. And the distances are so close. And at the same time, they are so wide with the wind backs, they are so deep with the Burnler mookers in behind. That's why you cannot be close because they push you here, push you here, and a lot of good players in the middle. And that's why it's difficult to, you know, to face the teams like they want the ball, they want to play with the ball. And we try to do it since the day one we arrived here. You've moved nicely onto Chelsea."

### Translate text

In [ ]:
# import request
# from config import DEEP_API_KEY

# def translate_text(text, target_lang="EN"):
#     """
#     Translate text to the target language using DeepL API.
#     """
#     url = "https://api-free.deepl.com/v2/translate"
#     payload = {
#         "auth_key":DEEP_API_KEY, 
#         "text":text, 
#         "target_lang":target_lang
#     }


In [ ]:
api_print = os.getenv("DEEP_API_KEY")
api_print

In [11]:
import deepl
import os
#from config import DEEP_API_KEY

def translate_text(text, target_lang, context=None):
    """
    Translate text using DeepL
    """
    DEEP_API_KEY = os.getenv("DEEP_API_KEY")
    translator = deepl.Translator(DEEP_API_KEY)

    result = translator.translate_text(text=text, 
                                        target_lang=target_lang, 
                                        context=context)
    return result.text



In [12]:
context = "This is a tactic football class video"
text_translated = translate_text(text, target_lang="PT-BR", context=context)

In [13]:
print(text)
print(text_translated)

 Why Chelsea plays so good? Because they have three centre defender closers, the two-holding midfielders, the relating move closers, the pocket closers, they have structured the five and two players so, so close. And the distances are so close. And at the same time, they are so wide with the wind backs, they are so deep with the Burnler mookers in behind. That's why you cannot be close because they push you here, push you here, and a lot of good players in the middle. And that's why it's difficult to, you know, to face the teams like they want the ball, they want to play with the ball. And we try to do it since the day one we arrived here. You've moved nicely onto Chelsea.
 Por que o Chelsea joga tão bem? Porque eles têm três zagueiros centrais mais próximos, dois meio-campistas de contenção, os que se movimentam em relação a eles, os que fecham o bolso, eles estruturaram os cinco e os dois jogadores de forma muito, muito próxima. E as distâncias são muito próximas. E, ao mesmo tempo, 

### TTS (Text To Speech)

In [14]:
import azure.cognitiveservices.speech as speechsdk

In [ ]:
AZURE_TTS_KEY = os.getenv("AZURE_TTS_KEY")
AZURE_REGION = os.getenv("AZURE_REGION")
print(AZURE_TTS_KEY)
print(AZURE_REGION)


In [19]:
def synthesize_speech(text:str, output_path:str, voice:str="pt-BR-MacerioMultilingualNeural"):
    """
    Convert text to speech
    """

    try:
        # Setting the auth
        AZURE_TTS_KEY = os.getenv("AZURE_TTS_KEY")
        AZURE_REGION = os.getenv("AZURE_REGION")
        speech_config = speechsdk.SpeechConfig(subscription=AZURE_TTS_KEY, region=AZURE_REGION)
        speech_config.speech_synthesis_voice_name = voice

        # Define the output
        audio_config = speechsdk.audio.AudioOutputConfig(filename=output_path)

        # Create the synthesizer
        synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)

        # Run synthesizer
        result = synthesizer.speak_text_async(text).get()

        # Check erro
        if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
            print(f"Audio in PT-BR created: {output_path}")
            return output_path
        elif result.reason == speechsdk.ResultReason.Canceled:
            cancellation_details = result.cancellation_details
            raise RuntimeError(f"Speech synthesis canceled: {cancellation_details.reason}. Error details: {cancellation_details.error_details}")

    except Exception as e:
        raise RuntimeError(f"An error when synthesizing speech: {e}")

In [20]:
output_audio_path = f"./video_data/traslated_audio_{video_name}.wav"
synthesize_speech(text_translated, output_audio_path)

Audio in PT-BR created: ./video_data/traslated_audio_Pep_Guardiola_explained_why_Chelsea_is_so_good.wav


'./video_data/traslated_audio_Pep_Guardiola_explained_why_Chelsea_is_so_good.wav'

### Merge Audio

In [21]:
output_video_path = f"./video_data/translated_{video_name}.mp4"
utils.merge_audio_with_video(video_path, output_audio_path, output_video_path)

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14 (GCC)
  configuration: --prefix=/usr --bindir=/usr/bin --datadir=/usr/share/ffmpeg --docdir=/usr/share/doc/ffmpeg --incdir=/usr/include/ffmpeg --libdir=/usr/lib64 --mandir=/usr/share/man --arch=x86_64 --optflags='-O2 -flto=auto -ffat-lto-objects -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wno-error=incompatible-pointer-types -Wp,-U_FORTIFY_SOURCE,-D_FORTIFY_SOURCE=3 -Wp,-D_GLIBCXX_ASSERTIONS -specs=/usr/lib/rpm/redhat/redhat-hardened-cc1 -fstack-protector-strong -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -m64 -march=x86-64 -mtune=generic -fasynchronous-unwind-tables -fstack-clash-protection -fcf-protection -mtls-dialect=gnu2 -fno-omit-frame-pointer -mno-omit-leaf-frame-pointer ' --extra-ldflags='-Wl,-z,relro -Wl,--as-needed -Wl,-z,pack-relative-relocs -Wl,-z,now -specs=/usr/lib/rpm/redhat/redhat-hardened-ld -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -Wl,--build-id

'./video_data/translated_Pep_Guardiola_explained_why_Chelsea_is_so_good.mp4'